<a href="https://colab.research.google.com/github/KaifAhmad1/deepfake/blob/main/Prompt_to_Animation_Video_Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -qU diffusers transformers accelerate torch
!pip install -qU opencv-python moviepy

In [ ]:
import os
import torch
import cv2
from diffusers import DiffusionPipeline
from google.colab import files, drive

# Mount Google Drive for storing outputs (Colab environment)
print("Mounting Google Drive...")
drive.mount('/content/drive')
OUTPUT_DIR = '/content/drive/MyDrive/video_outputs'
os.makedirs(OUTPUT_DIR, exist_ok=True)
print(f"Output directory set to: {OUTPUT_DIR}")

Mounting Google Drive...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Output directory set to: /content/drive/MyDrive/video_outputs


In [ ]:
# List of Diffusers video generation models with detailed storytelling prompts
DIFFUSERS_MODELS = [
    {
        "name": "cerspense/zeroscope_v2_576w",
        "prompt": "A cinematic animation that depicts a hopeful sunrise over a flourishing forest, symbolizing nature’s resilience and the urgency of environmental conservation."
    },
    {
        "name": "stabilityai/stable-video-diffusion-img2vid",
        "prompt": "A surreal yet inspirational narrative where a dog in a superhero outfit soars across a futuristic cityscape, merging fantasy with a call for unity."
    },
]

def generate_video_diffusers(model_name, prompt, output_path):
    """
    Generate video using the Diffusers pipeline.
    Parameters:
        model_name (str): The model identifier from Diffusers.
        prompt (str): Detailed text prompt describing the scene.
        output_path (str): The file path for saving the generated video.
    Returns:
        str: The path to the saved video file if successful; otherwise, None.
    """
    try:
        print(f"\nProcessing {model_name} with Diffusers pipeline using prompt:\n{prompt}")
        pipe = DiffusionPipeline.from_pretrained(
            model_name, torch_dtype=torch.float16, variant="fp16", use_auth_token=False
        )
        pipe = pipe.to("cuda")
        pipe.enable_model_cpu_offload()  # Optimize GPU memory utilization

        # Generate video frames from the given prompt
        video_frames = pipe(prompt, num_frames=16, height=576, width=1024).frames
        video_path = save_video(video_frames, output_path)
        print(f"Video successfully generated at {video_path}")
        return video_path
    except Exception as e:
        print(f"Error generating video with {model_name}: {str(e)}")
        return None

In [ ]:
def save_video(frames, output_path):
    """
    Save a list of frames as an MP4 video.
    Parameters:
        frames (list): A list of video frames (numpy arrays).
        output_path (str): The output file path (MP4 format).
    Returns:
        str: The file path where the video was saved.
    """
    height, width = frames[0].shape[:2]
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, 24.0, (width, height))
    for frame in frames:
        # Convert RGB to BGR color space for OpenCV
        frame_bgr = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
        out.write(frame_bgr)
    out.release()
    return output_path

In [ ]:
def main():
    """
    Main function to test and generate videos using Diffusers-based video generation models.
    """
    try:
        if not torch.cuda.is_available():
            raise RuntimeError("GPU not available. Please enable a GPU runtime in Colab.")

        for model in DIFFUSERS_MODELS:
            model_name = model["name"]
            prompt = model["prompt"]
            sanitized_name = model_name.replace('/', '_')
            output_path = os.path.join(OUTPUT_DIR, f"{sanitized_name}_output.mp4")
            video_path = generate_video_diffusers(model_name, prompt, output_path)
            if video_path:
                print(f"Generated video for {model_name} stored at {video_path}")
                files.download(video_path)
            else:
                print(f"Failed to generate video for {model_name}")

    except Exception as e:
        print(f"An error occurred in the main pipeline: {str(e)}")

if __name__ == "__main__":
    main()


Processing cerspense/zeroscope_v2_576w with Diffusers pipeline using prompt:
A cinematic animation that depicts a hopeful sunrise over a flourishing forest, symbolizing nature’s resilience and the urgency of environmental conservation.


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_index.json:   0%|          | 0.00/384 [00:00<?, ?B/s]

Error generating video with cerspense/zeroscope_v2_576w: You are trying to load model files of the `variant=fp16`, but no such modeling files are available. 
Failed to generate video for cerspense/zeroscope_v2_576w

Processing stabilityai/stable-video-diffusion-img2vid with Diffusers pipeline using prompt:
A surreal yet inspirational narrative where a dog in a superhero outfit soars across a futuristic cityscape, merging fantasy with a call for unity.


model_index.json:   0%|          | 0.00/498 [00:00<?, ?B/s]

...

preprocessor_config.json:   0%|          | 0.00/518 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/687 [00:00<?, ?B/s]

Keyword arguments {'use_auth_token': False} are not expected by StableVideoDiffusionPipeline and will be ignored.


Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

Error generating video with stabilityai/stable-video-diffusion-img2vid: `image` has to be of type `torch.Tensor` or `PIL.Image.Image` or `List[PIL.Image.Image]` but is <class 'str'>
Failed to generate video for stabilityai/stable-video-diffusion-img2vid
